In [21]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve

In [ ]:
def solve_GC(val, col_ind, row_ptr, b, x0, tol=1e-10):
    """
    Calculate the conjugate gradient of a sparse matrix.
    """
    n = len(row_ptr) - 1
    x = x0.copy()  # Initial guess
    r = np.zeros(n)  # Residual
    p = np.zeros(n)  # Search direction
    Ap = np.zeros(n)  # A*p

    # Initial residual r = b + A*x
    for i in range(n):
        for j in range(row_ptr[i], row_ptr[i + 1]):
            col = col_ind[j]
            value = val[j]
            r[i] += value * x[col]
        r[i] = b[i] + r[i]
    # Initial search direction p = r
    p = -r.copy()

    # Initial residual norm
    r_norm = sum(r[i] * r[i] for i in range(n))
    A_r = np.zeros(n)
    for i in range(n):
        for j in range(row_ptr[i], row_ptr[i + 1]):
            col = col_ind[j]
            value = val[j]
            A_r[i] += value * r[col]
    A_r_r = sum(r[i] * A_r[i] for i in range(n))
    if A_r_r == 0:
        return x
    q1 = r_norm / A_r_r
    x1 = x + q1 * r
    r1 = r - q1 * A_r

    x_x1 = x1 - x
    x_x1_norm = sum(x_x1[i] * x_x1[i] for i in range(n))
    x1_norm = sum(x1[i] * x1[i] for i in range(n))
    r1_norm = sum(r1[i] * r1[i] for i in range(n))

    while np.sqrt(r_norm) > tol and (np.sqrt(x_x1_norm)/np.sqrt(x1_norm) > tol):
        alpha = r1_norm / r_norm
        p = -r1 + alpha * p
        A_p = np.zeros(n)
        for i in range(n):
            for j in range(row_ptr[i], row_ptr[i + 1]):
                col = col_ind[j]
                value = val[j]
                A_p[i] += value * p[col]
        q1 = r1_norm / sum(p[i] * A_p[i] for i in range(n))
        x = x1
        x1 = x + q1 * p
        r = r1
        r1 = r - q1 * A_p
        r_norm = sum(r[i] * r[i] for i in range(n))
        r1_norm = sum(r1[i] * r1[i] for i in range(n))
        x_x1 = x1 - x
        x_x1_norm = sum(x_x1[i] * x_x1[i] for i in range(n))
        x1_norm = sum(x1[i] * x1[i] for i in range(n))
        r1_norm = sum(r1[i] * r1[i] for i in range(n))

    return x

In [23]:
# Coeficiente de difusão q(u) = 1 + u^2
def q_func(u):
    return 1.0 + u**2

# Vetor da solução exata u(x,y) = sin(πx)sin(πy)
def calculate_exact_solution(N, h):
    
    exact_u = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
            x = (i + 1) * h
            y = (j + 1) * h
            exact_u[i, j] = np.sin(np.pi * x) * np.sin(np.pi * y)
    
    return exact_u.flatten()

# Compara a solução numérica com a exata
def calculate_error(numerical_U, exact_U, h):
    
    error_vector = numerical_U - exact_U
    l2_error = np.sqrt(h**2 * np.sum(error_vector**2))
    max_error = np.max(np.abs(error_vector))
    
    return l2_error, max_error

# Cálculo do vetor do sistema não linear F(U)
def calculate_F(U_vector, N, h, source_f):
    
    u = np.zeros((N + 2, N + 2))
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    F = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            r, c = i + 1, j + 1
            
            u_c = u[r, c]
            u_n = u[r+1,c]
            u_s = u[r-1,c]
            u_e = u[r,c+1]
            u_w = u[r,c-1]
            
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            q_ip12j = (q_e + q_c) / 2.0
            q_im12j = (q_w + q_c) / 2.0
            q_ijp12 = (q_n + q_c) / 2.0
            q_ijm12 = (q_s + q_c) / 2.0

            x_term = q_ip12j * (u_e - u_c) - q_im12j * (u_c - u_w)
            y_term = q_ijp12 * (u_n - u_c) - q_ijm12 * (u_c - u_s)
            
            f_ij = source_f[i * N + j]
            F[i, j] = (x_term + y_term) / h**2 - f_ij
            
    return F.flatten()

# Monta a matriz Jacobiana J = dF/dU no formato CSR
def calculate_jacobian_csr(U_vector, N, h):

    u = np.zeros((N + 2, N + 2))
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    
    values = []
    column_indices = []
    row_pointers = [0]

    for i in range(N):
        for j in range(N):
            r, c = i + 1, j + 1
            
            u_c = u[r,c]
            u_n = u[r+1,c]
            u_s = u[r-1,c]
            u_e = u[r,c+1]
            u_w = u[r,c-1]
            
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            # dF/du_{i,j-1} (Oeste)
            if j > 0:
                val = (q_w + q_c)/2.0 - u_w * (u_c - u_w) 
                column_indices.append(i * N + (j - 1))
                values.append(val / h**2)

            # dF/du_{i-1,j} (Sul)
            if i > 0:
                val = (q_s + q_c)/2.0 - u_s * (u_c - u_s) 
                column_indices.append((i - 1) * N + j)
                values.append(val / h**2)
            
            # dF/du_{i,j} (Central)
            val_c = (-(q_e+q_w+q_n+q_s)/2.0 - 2*q_c) - u_c * ((u_e-u_c) + (u_w-u_c) + (u_n-u_c) + (u_s-u_c)) 
            column_indices.append(i * N + j)
            values.append(val_c / h**2)

            # dF/du_{i+1,j} (Norte)
            if i < N - 1:
                val = (q_n + q_c)/2.0 - u_n * (u_c - u_n) 
                column_indices.append((i + 1) * N + j)
                values.append(val / h**2)

            # dF/du_{i,j+1} (Leste)
            if j < N - 1:
                val = (q_e + q_c)/2.0 - u_e * (u_c - u_e) 
                column_indices.append(i * N + (j + 1))
                values.append(val / h**2)
            
            row_pointers.append(len(values))
            
    return np.array(values), np.array(column_indices), np.array(row_pointers)

# Resolve o sistema F(U)=0 com o Método de Newton
def solve_newton(initial_U, N, h, source_f, tol=1e-6, max_iter=50):
    
    U_k = np.copy(initial_U) 
    
    print("Iniciando Método de Newton...")

    for k in range(max_iter):
        
        F_k = calculate_F(U_k, N, h, source_f)
        J_values, J_cols, J_pointers = calculate_jacobian_csr(U_k, N, h)

        s_k = solve_GC(J_values, J_cols, J_pointers, -F_k, U_k)
        # J_csc = csc_matrix((J_values, J_cols, J_pointers), shape=(N*N, N*N))
        # s_k = spsolve(J_csc, -F_k) 

        U_k = U_k + s_k
        
        step_norm = np.linalg.norm(s_k, np.inf)
        print(f"Iteração {k+1:2d}: Norma do passo = {step_norm:.2e}")
        
        if step_norm < tol:
            print(f"\nConvergência atingida em {k+1} iterações.")
            return U_k

    print("\nMáximo de iterações excedido.")
    
    return U_k

In [24]:
# def solve_broyden(val, col_ind, row_ptr, b, x0, tol=1e-10, max_iter=100):
#     """
#     Calculate the Broyden's method for solving a system of equations.
#     """
#     n = len(row_ptr) - 1
#     x = x0.copy()  # Initial guess
#     r = [0] * n  # Residual
#     p = [0] * n  # Search direction
#     F = F(x) # Function value at x

#     A_inverse = np.zeros((n, n))  # Inverse of the Jacobian approximation
#     for i in range(n):
#         for j in range(row_ptr[i], row_ptr[i + 1]):
#             col = col_ind[j]
#             value = val[j]
#             A_inverse[i][col] = value
#     A_inverse = np.linalg.inv(A_inverse)  # Initial inverse of the Jacobian

#     s = np.zeros(n)  # Step
#     for i in range(n):
#         for j in range(row_ptr[i], row_ptr[i + 1]):
#             col = col_ind[j]
#             value = val[j]
#             s[i] += value * F[col]

In [25]:
N = 3
h = 1.0 / (N + 1)  # Grid spacing
source_f = np.array([2 * np.pi**2 * np.sin(np.pi * (i + 1) * h) * np.sin(np.pi * (j + 1) * h) 
                     for i in range(N) for j in range(N)])  # Fonte f(x,y)
initial_U = np.zeros((N, N)).flatten()  # Initial guess (zero everywhere)
x = solve_newton(initial_U, N, h, source_f)
print(f"Solução numérica obtida: {x}")

Iniciando Método de Newton...


TypeError: bad operand type for unary -: 'list'